# Compute salt change

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import cartopy.crs as ccrs
import xarray as xr
import cmocean.cm as cmocean
import glob
import matplotlib.path as mpath
import matplotlib.colors as col
import pandas as pd

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client


In [2]:
client = Client()
client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 250.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46485,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 250.00 GiB
Comm: tcp://127.0.0.1:32993,Total threads: 4
Dashboard: /proxy/39567/status,Memory: 35.71 GiB
Nanny: tcp://127.0.0.1:34397,


In [3]:
master_session = cc.database.create_session()

control = '01deg_jra55v13_ryf9091'

#session_name = '/g/data/e14/pc5520/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
session_name ='/g/data/x77/cy8964/databases/basal_melt_MOM5_v2.db'
#session_name = '/g/data/v45/akm157/model_data/access-om2/basal_melt_MOM5/basal_melt_MOM5.db'
basal_melt_session = cc.database.create_session(session_name)

# temperature of meltwater using Gade line:
basal = 'accessom2-GPC015'
iceberg = 'accessom2-GPC018'


In [4]:
expt = control
session = master_session

geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)


In [5]:
def shelf_mask_isobath(var):
    '''
    Masks ACCESS-OM2-01 variables by the region polewards of the 1000m isobath as computed using 
    a script contributed by Adele Morrison.
    Only to be used with ACCESS-OM2-0.1 output!
    '''
    contour_file = np.load('/g/data/ik11/grids/Antarctic_slope_contour_1000m.npz')
    
    shelf_mask = contour_file['contour_masked_above']
    yt_ocean = contour_file['yt_ocean']
    xt_ocean = contour_file['xt_ocean']
    
    # in this file the points along the isobath are given a positive value, the points outside (northwards) 
    # of the isobath are given a value of -100 and all the points on the continental shelf have a value of 0 
    # so we mask for the 0 values 
    shelf_mask[np.where(shelf_mask!=0)] = np.nan
    shelf_mask = shelf_mask+1
    shelf_map = np.nan_to_num(shelf_mask)
    shelf_mask = xr.DataArray(shelf_mask, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    shelf_map = xr.DataArray(shelf_map, coords = [('yt_ocean', yt_ocean), ('xt_ocean', xt_ocean)])
    
    # then we want to multiply the variable with the mask so we need to account for the shape of the mask. 
    # The mask uses a northern cutoff of 59S.
    masked_var = var.sel(yt_ocean = slice(-90, -59.03)) * shelf_mask
    return masked_var, shelf_map


In [6]:
def get_salt_snap(expt,session,start_time):
    salt_dummy = cc.querying.getvar(control, 'salt', master_session, n=1,frequency='1 monthly')
    if start_time == 2150:
        # control snapshot
        filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
        nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/accessom2_restart.nml'
    elif expt == basal:
        outputno = (start_time-2150)*6+996
        filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
        nml_path = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/accessom2_restart.nml'
    elif expt == iceberg:
        outputno = (start_time-2150)*12+995
        if start_time >= 2158:
            outputno = outputno-2
        if start_time <=2156:
            filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
            nml_path = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/accessom2_restart.nml'
        else:
            filepathsalt = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
            nml_path = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/accessom2_restart.nml'
    elif expt == control:
        if start_time == 2155:
            filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/ocean/ocean_thickness.res.nc'
            nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/accessom2_restart.nml'
            
        elif start_time == 2160:
            filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_thickness.res.nc'
            nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/accessom2_restart.nml'

        else:
            print('restarts not found, control only has 5 year increments available')
    else:
        print('experiment not found')
    restart1t = xr.open_dataset(filepathsalt)
    nml = pd.read_table(nml_path, skiprows=None, header=None)
    print('Time details')
    print(nml)
    # only 1 time snapshot:
    salt_init = restart1t.salt[0,...]
    # fix coords:
    salt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    salt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    salt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    salt_init = salt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(salt_init.Time.values)
    salt_init = salt_init.drop_vars('Time')

    restart1t = xr.open_dataset(filepathsalt)
    # only 1 time snapshot:
    temp_init = restart1t.temp[0,...]
    # fix coords:
    temp_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    temp_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    temp_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(temp_init.Time.values)
    temp_init = temp_init.drop_vars('Time')
    
    restart1z = xr.open_dataset(filepathdzt)
    # only 1 time snapshot:
    dzt_init = restart1z.rho_dzt[0,...]
    # fix coords:
    dzt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    dzt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    dzt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(dzt_init.Time.values)
    dzt_init = dzt_init.drop_vars('Time')

    return salt_init, temp_init, dzt_init

In [7]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = iceberg, years 2150 to 2155')
salt_init, temp_init,dzt_init = get_salt_snap(iceberg,basal_melt_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(iceberg,basal_melt_session,2155)

# #haline contraction
# depth = -salt_init.st_ocean.values # st_ocean value of the uppermost cell
# geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
# geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)
# ## extract coordinate arrays
# yt_ocean = SSS_PSU.yt_ocean.values
# xt_ocean = SSS_PSU.xt_ocean.values
# st_ocean = cc.querying.getvar(expt,'st_ocean',session,n=1).load()
# lon_t = geolon_t
# lat_t = geolat_t
# depth_tile = (lat_t*0+1)*depth
# pressure = xr.DataArray(p_from_z(depth_tile,lat_t), coords = [st_ocean,yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'pressure', attrs = {'units':'dbar'})
# # convert units to absolute salinity 
# SSS = xr.DataArray(SA_from_SP(salt_init,pressure,lon_t,lat_t), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['time','yt_ocean', 'xt_ocean'], name = 'sea surface salinity', attrs = {'units':'Absolute Salinity (g/kg)'})
# # Compute salt transformation (no density binning)
# haline_contraction_init = xr.DataArray(beta(salt_init, temp_init-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})
# haline_contraction_end = xr.DataArray(beta(salt_end, temp_end-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})

# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


# FW_content_change = (((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area).sum()
# print('Net FW content change over this period =',salt_content_change.values/1e12,'x 10^12 kg m^3')

# #shelf value
# FW_shelf, shelf_mask = shelf_mask_isobath((((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area))
# FW_shelf = FW_shelf.sum()
# print('Net FW content change over shelf in this period =',FW_shelf.values/1e12,'x 10^12 kg m^3')


expt = iceberg, years 2150 to 2155
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2155-01-01T00:00:00
3                                                /
Net salt content change over this period = -486.1717180934823 x 10^12 kg
Net salt content change over shelf in this period = -81.23493579632137 x 10^12 kg


In [8]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = basal, years 2150 to 2155')
salt_init, temp_init,dzt_init = get_salt_snap(basal, basal_melt_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(basal, basal_melt_session,2155)


# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


expt = basal, years 2150 to 2155
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2155-01-01T00:00:00
3                                                /
Net salt content change over this period = -255.13147260714985 x 10^12 kg
Net salt content change over shelf in this period = -66.29520426598087 x 10^12 kg


In [12]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = control, years 2150 to 2155')
salt_init, temp_init,dzt_init = get_salt_snap(control, master_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(control, master_session,2155)


# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


expt = control, years 2150 to 2155
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2155-01-01T00:00:00
3                                                /
Net salt content change over this period = -0.5121560807162145 x 10^12 kg
Net salt content change over shelf in this period = -0.010199346850187585 x 10^12 kg


In [14]:
((salt_init*dzt_init).sum('st_ocean')*area).sum().values

array(4.76695073e+22)

In [15]:
salt_init.sum().values

array(1.9888873e+45)

# 10 years

In [8]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = iceberg, years 2150 to 2160')
salt_init, temp_init,dzt_init = get_salt_snap(iceberg,basal_melt_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(iceberg,basal_melt_session,2160)

# #haline contraction
# depth = -salt_init.st_ocean.values # st_ocean value of the uppermost cell
# geolon_t = cc.querying.getvar(expt,'geolon_t',session, n=1)
# geolat_t = cc.querying.getvar(expt,'geolat_t',session, n=1)
# ## extract coordinate arrays
# yt_ocean = SSS_PSU.yt_ocean.values
# xt_ocean = SSS_PSU.xt_ocean.values
# st_ocean = cc.querying.getvar(expt,'st_ocean',session,n=1).load()
# lon_t = geolon_t
# lat_t = geolat_t
# depth_tile = (lat_t*0+1)*depth
# pressure = xr.DataArray(p_from_z(depth_tile,lat_t), coords = [st_ocean,yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'pressure', attrs = {'units':'dbar'})
# # convert units to absolute salinity 
# SSS = xr.DataArray(SA_from_SP(salt_init,pressure,lon_t,lat_t), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['time','yt_ocean', 'xt_ocean'], name = 'sea surface salinity', attrs = {'units':'Absolute Salinity (g/kg)'})
# # Compute salt transformation (no density binning)
# haline_contraction_init = xr.DataArray(beta(salt_init, temp_init-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})
# haline_contraction_end = xr.DataArray(beta(salt_end, temp_end-273.15, pressure), coords = [st_ocean, yt_ocean, xt_ocean], dims = ['st_ocean','yt_ocean', 'xt_ocean'], name = 'saline contraction coefficient (constant conservative temp)', attrs = {'units':'kg/g'})

# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


# FW_content_change = (((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area).sum()
# print('Net FW content change over this period =',salt_content_change.values/1e12,'x 10^12 kg m^3')

# #shelf value
# FW_shelf, shelf_mask = shelf_mask_isobath((((haline_contraction_end*salt_end*dzt_end).sum('st_ocean') - (haline_contraction_init*salt_init*dzt_init).sum('st_ocean'))*area))
# FW_shelf = FW_shelf.sum()
# print('Net FW content change over shelf in this period =',FW_shelf.values/1e12,'x 10^12 kg m^3')


expt = iceberg, years 2150 to 2160
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2160-01-01T00:00:00
3                                                /
Net salt content change over this period = -956.3723568537245 x 10^12 kg
Net salt content change over shelf in this period = -107.32338079912904 x 10^12 kg


In [11]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = basal, years 2150 to 2160')
salt_init, temp_init,dzt_init = get_salt_snap(basal, basal_melt_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(basal, basal_melt_session,2160)


# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


expt = basal, years 2150 to 2160
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2160-01-01T00:00:00
3                                                /
Net salt content change over this period = -501.6298392196165 x 10^12 kg
Net salt content change over shelf in this period = -85.43253399267067 x 10^12 kg


In [12]:
# compute salt content change:

# J / kg / degC
c_p = 3992.1
rho0 = 1035.0
area = cc.querying.getvar(control, 'area_t', master_session, n=1).load()

print('expt = control, years 2150 to 2160')
salt_init, temp_init,dzt_init = get_salt_snap(control, master_session,2150)
salt_end, temp_end,dzt_end = get_salt_snap(control, master_session,2160)


# salt in g/kg, multiply by kg in cell = rho0*area_t*dzt, gives g of salt, then divide by 1000 gives kg of salt.

# there is a factor of rho_0 in dzt so no need to include:
salt_content_change = (((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area).sum()
print('Net salt content change over this period =',salt_content_change.values/1e3/1e12,'x 10^12 kg')

#shelf value
salt_shelf, shelf_mask = shelf_mask_isobath((((salt_end*dzt_end).sum('st_ocean') - (salt_init*dzt_init).sum('st_ocean'))*area))
salt_shelf = salt_shelf.sum()
print('Net salt content change over shelf in this period =',salt_shelf.values/1e3/1e12,'x 10^12 kg')


expt = control, years 2150 to 2160
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2150-01-01T00:00:00
3                                                /
Time details
                                                 0
0                                 &DO_NOT_EDIT_NML
1   FORCING_CUR_DATE        = 1900-01-01T00:00:00,
2            EXP_CUR_DATE    = 2160-01-01T00:00:00
3                                                /
Net salt content change over this period = -0.7031921257098995 x 10^12 kg
Net salt content change over shelf in this period = 0.34887606037682795 x 10^12 kg


## Try turn into salt budget more accurately?

In [13]:
def get_salt_snap(expt,session,start_time):
    salt_dummy = cc.querying.getvar(control, 'salt', master_session, n=1,frequency='1 monthly')
    if start_time == 2150:
        # control snapshot
        filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/ocean/ocean_thickness.res.nc'
        nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart995/accessom2_restart.nml'
    elif expt == basal:
        outputno = (start_time-2150)*6+996
        filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
        nml_path = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC015/restarts/restart'+str(outputno)+'/accessom2_restart.nml'
    elif expt == iceberg:
        outputno = (start_time-2150)*12+995
        #filepathsalt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        #filepathdzt = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
        #nml_path = '/g/data/ik11/outputs/access-om2-01/basal_melt_outputs/accessom2-GPC018/restarts/restart'+str(outputno)+'/accessom2_restart.nml'
        
        filepathsalt = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/ocean/ocean_temp_salt.res.nc'
        filepathdzt = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/ocean/ocean_thickness.res.nc'
        nml_path = '/home/552/pc5520/access-om2/control/01deg_jra55v13_ryf9091_rerun_for_easterlies/archive/restart'+str(outputno)+'/accessom2_restart.nml'
    elif expt == control:
        if start_time == 2155:
            filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/ocean/ocean_thickness.res.nc'
            nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1015/accessom2_restart.nml'
            
        elif start_time == 2160:
            filepathsalt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_temp_salt.res.nc'
            filepathdzt = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/ocean/ocean_thickness.res.nc'
            nml_path = '/scratch/x77/cy8964/access-om2/archive/'+control+'/restart1035/accessom2_restart.nml'

        else:
            print('restarts not found, control only has 5 year increments available')
    else:
        print('experiment not found')
    restart1t = xr.open_dataset(filepathsalt)
    nml = pd.read_table(nml_path, skiprows=None, header=None)
    print('Time details')
    print(nml)
    # only 1 time snapshot:
    salt_init = restart1t.salt[0,...]
    # fix coords:
    salt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    salt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    salt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    salt_init = salt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(salt_init.Time.values)
    salt_init = salt_init.drop_vars('Time')

    restart1t = xr.open_dataset(filepathsalt)
    # only 1 time snapshot:
    temp_init = restart1t.temp[0,...]
    # fix coords:
    temp_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    temp_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    temp_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    temp_init = temp_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(temp_init.Time.values)
    temp_init = temp_init.drop_vars('Time')
    
    restart1z = xr.open_dataset(filepathdzt)
    # only 1 time snapshot:
    dzt_init = restart1z.rho_dzt[0,...]
    # fix coords:
    dzt_init.coords['zaxis_1'] = salt_dummy['st_ocean'].values
    dzt_init.coords['yaxis_1'] = salt_dummy['yt_ocean'].values
    dzt_init.coords['xaxis_1'] = salt_dummy['xt_ocean'].values
    dzt_init = dzt_init.rename(({'zaxis_1':'st_ocean', 'yaxis_1':'yt_ocean', 'xaxis_1':'xt_ocean'}))
    #print(dzt_init.Time.values)
    dzt_init = dzt_init.drop_vars('Time')

    return salt_init, temp_init, dzt_init